In [3]:
run_id, task_id, flow_name, step_name = None, None, None, None

In [ ]:
import numpy as np, pandas as pd, torch, IProgress
from glob import glob
import pandas as pd
import shutil, os
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import yaml
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
from sklearn import datasets, model_selection
from sklearn.model_selection import train_test_split
from metaflow import FlowSpec, Flow, step, IncludeFile, card, Parameter
import os.path
from os.path import exists
train_df = pd.read_csv('train.csv',nrows=500)

train_df['image_path'] = f'../512data/train/'+(train_df).image_id+('.png')

train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

print("Training on",(train_df).image_id.nunique(),"images")

#Pre-Processing
train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
#print(train_df.head())

features = ['x_min', 'y_min', 'x_max', 'y_max', 'x_mid', 'y_mid', 'w', 'h', 'area']
X = train_df[features]
y = train_df['class_id']
#(X).shape, (y).shape

class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
#print(classes)

train_data, val_data = model_selection.train_test_split(train_df, test_size = 0.30)
train_df.head()

train_files = []
val_files   = []
val_files += list(val_data.image_path.unique())
train_files += list(train_data.image_path.unique())

os.makedirs('labels/train', exist_ok = True)
os.makedirs('labels/val', exist_ok = True)
os.makedirs('images/train', exist_ok = True)
os.makedirs('images/val', exist_ok = True)
label_dir = '../labels'

for file in (train_files):
    shutil.copy(file, 'images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), 'labels/train')

for file in (val_files):
    shutil.copy(file, 'images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), 'labels/val')


with open('./train.txt', 'w') as f:
    for path in glob('./images/train/*'):
        f.write(path+'\n')

with open('./val.txt', 'w') as f:
    for path in glob('./images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( '../vinbigdata/train.txt'),
    val   =  join( '../vinbigdata/val.txt' ),
    nc    = 14,
    names = classes
    )

with open(join( '../yolov5', 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( '../yolov5', 'vinbigdata.yaml'), 'r')

os.system('cd .. && cd yolov5 && python train.py --img 128 --batch 16 --epochs 2 --data vinbigdata.yaml --weights .weights/yolov5x.pt --cache')